## 검증 세트 (validation set)
테스트를 마지막에 한 번만 사용하기 위해 테스트 셋 대신 사용하는 데이터 셋
<br> 테스트셋을 사용하지 않고도 모델이 과대적합인지 과소적합인지 알 수 있음

In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

In [4]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state = 42)

In [5]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [6]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


<br>

## 교차 검증 (cross validation)
검증 세트를 떼어 내어 평가하는 과정을 여러번 반복
<br> k-fold cross validation : 훈련 세트를 k개의 부분으로 나누어 교차 검증을 수행
<br> 주로 5-폴드나 10-폴드를 이용
<br>
<br>사이킷런의 cross_validate()라는 교차 검증 함수를 이용
<br> 평가할 모델 객체를 첫 번째 매개 변수로, 검증 셋을 떼어 내지 않은 훈련 셋을 두 번째 매개 변수로 전달
<br> 디폴트로 5-fold 교차 검증을 시행함

In [7]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
scores

{'fit_time': array([0.00498486, 0.00398374, 0.00498223, 0.0049839 , 0.00597954]),
 'score_time': array([0.       , 0.0010004, 0.       , 0.       , 0.       ]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

위 함수는 fit_time, score_time, test_score 키를 가진 딕셔너리를 반환
<br> 처음 두 개는 각각 모델을 훈련하는 시간과 검증하는 시간을 의미
<br> test_score 키를 평균하면 ""검증 폴드의 점수""를 구할 수 있음

In [8]:
import numpy as np
np.mean(scores['test_score'])

0.855300214703487

### 분할기 (splitter)
사이킷런의 분할기는 교차 검증에서 폴드를 어떻게 나눌지 결정해줌 = 훈련 셋을 섞어줌
<br>
<br> cross_validate() 함수는 회귀 모델일 경우 KFold 분할기를 사용, 분류 모델일 경우 StratifiedKFold를 사용

In [9]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
np.mean(scores['test_score'])

0.855300214703487

분할기를 사용하는 경우는 다음과 같음

In [10]:
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42) #n_splits 매개변수는 k-fold의 k를 결정
scores = cross_validate(dt, train_input, train_target, cv = splitter)
np.mean(scores['test_score'])

0.8574181117533719

<br>

## 하이퍼파라미터 튜닝(hyper-parameter tuning)
<br> 하이퍼파라미터 튜닝 방법
1. 라이브러리가 제공하는 기본값을 그대로 사용해 모델 훈련
2. 검증 셋의 점수나 교차 검증을 통해 매개변수를 바꿔봄 <br>

max_depth의 최적값은 min_smaples_split 매개변수의 값에 따라 변함. 따라서 두 매개변수를 동시에 바꿔가며 최적값을 찾아야 함
- max_depth : 트리가 성장할 최대 깊이를 지정. 기본값은 None으로 리프 노드가 순수하거나 min_saples_split 보다 샘플 개수가 적을 때까지 성장
- min_samples_split : 노드를 나누기 위한 최소 샘플 개수. 기본값은 2<br>

사이킷런의 그리드 서치(Grid Search)를 사용, GridSearchCV 클래스
- 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행
- 별도로 cross_validate() 함수 호출 필요 없음

In [11]:
#실습1) 기본 매개변수를 사용한 결정 트리 모델에서 min_impurity_decrease 매개변수의 최적값 찾기
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1, refit=True)

GridSearchCV의 cv 매개변수 기본값은 5
<br> min_impurity_decrease 값마다 5-폴드 교차 검증을 수행하므로, 5*5 = 25개의 모델을 훈련함
<br>
<br> GridSearchCV의 n_jobs 매개변수의 기본값은 1
<br> 병렬에 실행에 사용할 CPU 코어 수를 지정, -1로 지정하면 시스템에 있는 모든 코어를 사용함

In [12]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

교차 검증에서 최적의 하이퍼파리미터를 찾으면 전체 훈련 셋으로 모델을 다시 만들어야함
<br> 사이킷런의 그리드 서치는 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈렬 셋에서 자동으로 다시 모델을 훈련함
<br>
<br> 이 모델은 gs 객체의 best_estimater_ 속성에 저장되어 있음
<br> 최적의 매개변수는 gs 객체의 best_params 속성에 저장되어 있음
<br> 각 매개변수에서 수행한 교차 검증의 평균 점수는 gs 객체의 cv_results_ 속성의 'mean_test_score'키에 저장되어 있음
<br> 각 매개변수에서 수행한 교차 검증에 해당하는 매개변수 값은 gs 객체의 cv_results_ 속성의 'params'키에 저장되어 있음

In [13]:
dt = gs.best_estimator_
dt.score(train_input, train_target)

0.9615162593804117

In [14]:
gs.best_params_

{'min_impurity_decrease': 0.0001}

In [15]:
gs.cv_results_['mean_test_score']

array([0.86819297, 0.86453617, 0.86492226, 0.86780891, 0.86761605])

In [16]:
# 넘파이의 argmax() 함수를 이용해 가장 큰 값의 인덱스를 추출
best_index = np.argmax(gs.cv_results_['mean_test_score'])
gs.cv_results_['params'][best_index]

{'min_impurity_decrease': 0.0001}

### 하이퍼파라미터 튜닝 방법 정리
1. 탐색할 매개변수 지정
2. 훈련 셋에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾음. 그 조합은 그리드 서치 객체에 저장됨
3. 그리드 서치는 최상의 매개변수에서 (교차 검증에 사용한 훈련 셋이 아니라) 전체 훈련 셋을 사용해 최종 모델을 훈련. 이 모델도 그리드 서치 객체에 저장됨

In [17]:
#실습2) max_depth, min_impurity_decrease,min_samples_split 매개변수의 최적값 찾기
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
         'max_depth' : range(5, 20, 1),
         'min_samples_split' : range(2, 100, 10)}

위 매개변수로 수행할 교차 검증 횟수는 9x15x10

In [18]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [19]:
#최상의 매개 변수 조합
gs.best_params_

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}

In [20]:
#최상의 교차 검증 점수
np.max(gs.cv_results_['mean_test_score'])

0.8683865773302731

### 랜덤 서치 (random search)
랜덤 서치에 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달

In [21]:
from scipy.stats import uniform, randint

In [22]:
# 난수발생기와 유사한 역할
# randint는 균등 분포에서 정숫값 샘플링
rgen = randint(0, 10)
rgen.rvs(10)

array([1, 3, 5, 0, 3, 3, 7, 3, 3, 1], dtype=int64)

In [23]:
np.unique(rgen.rvs(1000), return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64),
 array([ 95,  95,  92, 108, 100,  94, 103, 119, 115,  79], dtype=int64))

In [24]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.03643013, 0.7777696 , 0.65342526, 0.46926798, 0.54540058,
       0.6772472 , 0.72761904, 0.20556638, 0.12370434, 0.91811306])

- 랜덤서치로 매개변수 도출
<br>min_samples_leaf 매개변수 추가.
<br>어떤 노드가 분할하여 만들어질 자식 노드의 샘플 수가 이 값보다 작을 경우 분할하지 않음

In [25]:
params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
         'max_depth' : randint(20,50),
         'min_samples_split' : randint(2, 25),
         'min_samples_leaf' : randint(1, 25)}

샘플링 횟수는 사이킷런의 랜덤 서치 클래스인 RandomizedSearchCV의 n_iter 매개변수에 지정

In [26]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params,
                       n_iter = 100, n_jobs = -1, random_state = 42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002518F0FCD90>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000251FFEA8490>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002518F0FE350>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002518F0FD690>},
                   random_state=42)

In [27]:
#최적의 매개변수 조합
gs.best_params_

{'max_depth': 39,
 'min_impurity_decrease': 0.00034102546602601173,
 'min_samples_leaf': 7,
 'min_samples_split': 13}

In [28]:
#최고의 교차 검증 점수
np.max(gs.cv_results_['mean_test_score'])

0.8695428296438884

In [29]:
#최적의 모델은 전체 훈련 세트로 훈련되어 best_estimator_ 속성에 저장되어 있음
dt = gs.best_estimator_
dt.score(test_input, test_target)

0.86